In [ ]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.optimize as spop
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# GARCH(2,2) Model
$$ a_t = ε_t\sqrt{ω+α1a_{(t-1)}^2+α2a_{(t-2)}^2+β1σ_{(t-1)}^2+β1σ_{(t-2)}^2}\quad$$
$$ α_0,α_1 ~N(0,1) $$
$$ σ_0=1,σ_1=1$$
$$ ε_t  ~N(0,1)$$

# Stock Volatility

In [ ]:
start = datetime(2009,1,1)
end = datetime(2021,9,5)

In [ ]:
HD_data = yf.Ticker('HD')
HD = HD_data.history(period='1m', start=start, end=end)

In [ ]:
returns = HD.Close.pct_change().dropna() 
returns.head()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(returns)
plt.ylabel('Pct Retrun',fontsize=16)
plt.title('HD',fontsize=20)
plt.show()

# PACF

In [ ]:
plot_pacf(returns**2)
plt.show()

# GARCH(2,2)

In [ ]:
model = arch_model(returns, p=2, q=2)
model_fit = model.fit()

In [ ]:
model_fit.summary()

Due to all beta are insignifianct, we switch to the ARCH(2) model (or GARCH(2,0)).

In [ ]:
model = arch_model(returns, p=2, q=0)
model_fit = model.fit()

In [ ]:
model_fit.summary()

# Evaluation : Rolling forcasting orgin

In [ ]:
rolling_predictions = []
test_size = 120
for i in range(test_size):
  train = returns[:-(test_size-i)]
  model = arch_model(train, p=3, q=0)
  model_fit = model.fit(disp='off')
  pred = model_fit.forecast(horizon=1)
  rolling_predictions.append(np.sqrt(pred.variance.values[-1,:][0]))

In [ ]:
rolling_predictions = pd.Series(rolling_predictions, index = returns.index[-120:]) 

In [ ]:
plt.figure(figsize=(10,4))
true = plt.plot(returns[-120:])
preds = plt.plot(rolling_predictions)
plt.title('Rolling Forecast', fontsize=20)
plt.legend(['True Retrun','Predicted Volatility'], fontsize= 16)
plt.show()

In [ ]:
train = returns
model = arch_model(train,p=2, q=0)
model_fit = model.fit(disp='off')

In [ ]:
pred = model_fit.forecast(horizon=4)
future_dates = [returns.index[-1]+timedelta(weeks= 4*i) for i in range(1,5,1)]
pred = pd.Series(np.sqrt(pred.variance.values[-1,:]),index = future_dates)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(pred)
plt.title('Future', fontsize=20)
plt.show()